# OpenReviewPy Tutorial


This tutorial will demonstrate how to use the OpenReviewPy library to manage your conference.




### Getting Started

Before you begin, make sure that you've installed the latest version of OpenReviewPy using pip:

```
pip install openreviewpy
```


In [ ]:
# Start by importing all four OpenReviewPy classes
from openreview import *

### OpenReviewPy classes
OpenReviewPy includes four classes:
- **Client**: used to interact with the OpenReview servers.
- **Group**: represents an OpenReview *entity* 
    - E.g. organizations, conferences, individuals, reviewers, program committees, etc.
- **Note**: represents a *record* within OpenReview 
    - E.g. paper submissions, comments, reviews, etc.
- **Invitation**: represents an *invitation for users to submit notes*
    - E.g. a call for paper submissions, an invitation open to the public to submit comments, an invitation to only the program committee to submit a review, etc.

In [ ]:
# Let's start by initializing the Client with our username, password, and base URL
openreview_client = Client(username='your_username@mail.com',password='your_password',baseurl='http://openreview.net')